<a href="https://colab.research.google.com/github/DiegoRomanCortes/AED/blob/master/Tarea_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Enunciado](https://www.u-cursos.cl/ingenieria/2020/1/CC3001/1/tareas/r/1_CC3001-Tarea2-Diagrama_de_estados.pdf)

#Introducción
Este notebook tiene el objetivo de ilustrar el uso de diagramas de estado en la programación de una calculador simple tipo Maple o Matlab. Las condiciones son restrictivas, lo que facilita la implementación del mismo. Hay dos comandos, o bien asignar en la memoria una variable (incluyendo operaciones matemáticas) o bien imprimir una variable ya registrada en pantalla. Las operaciones permitidas son $+$, $-$, $*$, $/$ y ^. Sólo se operarán números enteros (la división por lo mismo será entera). Las variables pueden partir con una letra e ir seguidas por letras, números o '_'. Además, no habrá paréntesis, por lo que se evalurán las expresiones de izquierda a derecha (a partir del $=$).

#Diagramas de Estado

El diagrama de estado más simple es el que se dibuja a nivel macro. Existen dos estado que ve el usuario: In (que es cuando se le pide ingresar un comando) y Out (el resultado del comando que ha sido escrito).


![Entrada y Salida](https://github.com/DiegoRomanCortes/AED/blob/master/Entrada-Salida.png?raw=true)

Todo el aparataje está contenido en el estado In, pues Out sólo imprime en pantalla lo procesado por In. Así, el lector comienza a leer el String proporcionado por el usuario. El primer caracter puede ser el inicio de una variable, el inicio de un numero, un espacio u otro caracter. Como el comando tipo es del tipo 
```
variable = expresion
``` 
y si se omite el '=' se entiende que se quiere imprimir en pantalla la variable, se necesita que el primer Estado no Espacio al que ingrese el lector sea Variable, de lo contrario será un error de sintaxis.

Acto seguido, el lector debe asegurarse que el siguiente Estado no Espacio sea una Operacion (en concreto, el caracter debe ser '='). Tanto si el comando era de asignación como si era de impresión, cualquier otro Estado debe proporionar un error de sintaxis.

Se almacena la variable de la izquierda como
```
izquierda
```
A nivel general, el programa funciona almacenando la parte derecha de la igualdad en una variable llamada


```
derecha
```
la cual es asignada al haber llegado a un Estado Espacio o a un Estado Operacion (operación incluye las operaciones anteriormente detalladas y '=') y al tener en memoria otra variable llamada 

```
aux
```
que simboliza el lado derecho de una operación.

El lector termina cuando detecta el último Estado Espacio (el programa se asegura de esto concatenando un espacio al String original) y hace la asignación descrita en el párrafo anterior.

Finalmente, se añade al diccionario 

```
dic
```
la variable por definir de la forma:

```
dic[izquierda] = derecha
```
Si el comando era de impresión, simplemente se obvia el cálculo de 

```
derecha
```
y se imprime el valor que ya tenía asignada la variable (que si está mal escrita arroja un error de variable indefinida).



![Lector](https://github.com/DiegoRomanCortes/AED/blob/master/DiagramaEstado.png?raw=true)

Se llega hasta el Estado Variable si el caracter es una letra. No se sale del Estado Variable mientras el caracter no deje de ser un caracter permitido para la escritura de variables (una letra, un número o un guión bajo). Cualquier caracter distinto arrojará un error de sintaxis. Se sale del Estado Variable si el caracter es un espacio (entrado en el Estado Espacio) o si es una operacion. Si 

```
variable
```
está en memoria, se reemplaza su valor y se añade a 

```
derecha
```
o a

```
aux
```


dependiendo de 

```
operacion
```
Si no está en memoria, existen dos casos: O es una variable por definir (es decir, se está en el lado izquierdo de la igualdad), o es un error de variable indefinida (si se está en el lado derecho).

Para el estado Numero, el proceso es análogo a excepción de que no se arroja el error de variable indefinida ni se guarda en memoria una variable. Esto es, al salir del Estado Número, se reemplaza su valor y se añade dependiendo de la operación.

El Estado Error de Sintaxis se activa por descarte de los casos contemplados.

El Error de Variable Indefinida detiene de inmediato la ejecución del ciclo e imprime la variable que no se encuentra en memoria.



In [0]:
dic = {}

Enteros = ['0', '1', '2', '3', '4', '5', '6', '7', '8' ,'9']

Letras = ['a','b','c','d','e','f','g',\
          'h','i','j','k','l','m','n',\
          'o','p','q','r','s','t','u',\
          'v','w','x','y','z',\
          'A', 'B', 'C', 'D', 'E', 'F',\
          'G', 'H', 'I', 'J', 'K', 'L', \
          'M', 'N', 'O', 'P', 'Q', 'R', \
          'S', 'T', 'U', 'V', 'W', 'X', \
          'Y', 'Z']

Operaciones = ['+','-','*','/','^','=']

Caracteres = Enteros + Letras + ['_']

def pedirEntrada():
  Entrada = input()
  return Entrada

def leerEntrada(entrada):
  entrada += ' '

  Estado = 'Inicio' #no es importante pero sive para inicializar Estado
  Subestado = 'Izq' #a la izquierda de un '='

  variable = '' #aquí se almanenarán temporalmente las variables hasta llegar a una operacion
  numero = '' #lo mismo pero para numeros enteros
  operacion = '' #almacena la última operación leída para operarla después
  error = '' #guarda la entrada desde donde se detecta error de sintaxis

  izquierda = '' #variable por definir si es que es un comando de asignación, también variable de printeo
  derecha = '' #expresion 
  aux = '' #almacena la parte a la derecha de una operacion

  errvarindef = 'ERROR: variable indefinida "'
  errsintaxis = 'ERROR: al procesar "'
  
  DeboLeer = True
  
  for char in entrada: #se procesan los caracter de uno en uno
    if Estado == 'ErrorSintaxis':
        error += char
        continue
  #primero analicemos el primer caracter de cada Estado
    if char in Letras and variable == '' and Estado in ['Inicio', 'Operacion', 'Espacio'] and DeboLeer: #inicio de una variable
      Estado = 'Variable'
      variable += char
    elif char in Enteros and numero == '' and Estado in ['Operacion', 'Espacio'] and DeboLeer: #inicio de un numero entero
      Estado = 'Numero'
      numero += char
    elif char in Operaciones and Estado in ['Variable', 'Numero', 'Espacio']:
      if char == '=' and operacion != '':
          Estado = 'ErrorSintaxis'
          error += char
          continue
      Estado = 'Operacion'
      DeboLeer = True
    elif char == ' ':
      Estado = 'Espacio'
    #ahora veamos que pasa en cada estado 
    elif char in Caracteres and Estado == 'Variable':
      variable += char
    elif char in Enteros and Estado == 'Numero':
      numero += char
    elif char == ' ' and Estado == 'Espacio':
      pass
    else:
      Estado = 'ErrorSintaxis'
      error += char
      continue
    #ahora, qué pasa al cambiar de estado
    if Estado == 'Espacio' or Estado == 'Operacion':
      if Subestado == 'Izq':
        if char == '=':
            Subestado = 'Der'
            izquierda = variable
            variable = ''
            operacion = char
        elif Estado == 'Espacio':
            pass
        else:
            Estado = 'ErrorSintaxis'
            error += char
            continue
      else: #Estamos en la derecha
        if variable != '':
          if variable in dic:
            aux = dic[variable]
            variable = ''
            DeboLeer = False
          else:
            print(errvarindef, variable, '"')
            return
        elif numero != '':
          aux = int(numero)
          numero = ''
          DeboLeer = False
        if operacion == '=' and derecha == '':
          derecha = aux
          aux = ''
    if aux != '':
        if operacion == '+':
            derecha = derecha + aux
        elif operacion == '-':
            derecha = derecha - aux
        elif operacion == '*':
            derecha = derecha * aux
        elif operacion == '/':
            derecha = derecha // aux
        elif operacion == '^':
            derecha = derecha ** aux
        aux = ''
    if Estado == 'Operacion' and char != '=':
        operacion = char
  if Estado == 'ErrorSintaxis':
    print(errsintaxis, error, '"')
    return
  elif Subestado == 'Der': #asignacion
    dic[izquierda] = derecha
  else: #no cambio el subestado, por tanto es printeo
    izquierda = variable
    if variable in dic:
      pass
    else:
      print(errvarindef, izquierda, '"')
      return
  print(dic[izquierda])

In [5]:
print('Esta es una calculadora simple de operaciones entre números enteros')
print('Las operaciones permitidas son +, -, *, /, ^')
print('Usted puede asignar una variable o imprimir el valor de una almacenada en memoria')
print('Para salir, escriba "salir" en la entrada\n')
while True:
  ent = pedirEntrada()
  if ent == 'salir': break
  leerEntrada(ent)

Esta es una calculadora simple de operaciones entre números enteros
Las operaciones permitidas son +, -, *, /, ^
Usted puede asignar una variable o imprimir el valor de una almacenada en memoria
Para salir, escriba "salir" en la entrada

salir
